## Import Libraries

In [23]:

import os
import json
import requests
import itertools
import numpy as np
import pandas as pd
from pprint import pprint
from sqlalchemy.orm import Session
from sqlalchemy import create_engine


## Connect to Postgress SQL DB

In [31]:

pg_user = 'postgres'
pg_password = 'mobo0056'
db_name = 'Enterprises'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')



#### Get table Names

In [32]:

engine.table_names()


['NAICS', 'BUSINESSES', 'STATES']

#### Create dataframes from SQL Query

In [98]:

bus_df = pd.read_sql_query('select * from "BUSINESSES"', con=engine)
naics_df = pd.read_sql_query('select * from "NAICS"', con=engine)
states_df = pd.read_sql_query('select * from "STATES"', con=engine)


#### Refine Dataframesto only include relevant columns

In [99]:

business_df = bus_df[[ 'STATE DESCRIPTION', 'NAICS CODE', 'ENTERPRISE EMPLOYMENT SIZE 2', 'NUMBER OF FIRMS', 'NUMBER OF ESTABLISHMENTS', 'EMPLOYMENT', 'ANNUAL PAYROLL ($1,000)', 'YEAR']].copy()
business_df = business_df.rename(columns = {'ENTERPRISE EMPLOYMENT SIZE 2': 'ENTERPRISE EMPLOYMENT SIZE' })

states_df = states_df.rename(columns={'STATE':'STATE DESCRIPTION'})
states_df = states_df[["STATE DESCRIPTION","STATE CODE"]]


#### Convert Payroll column to real value from rounded value

In [100]:

business_df['ANNUAL PAYROLL'] = business_df['ANNUAL PAYROLL ($1,000)'] * 1000


#### Delete Original Payroll Column

In [101]:

del business_df['ANNUAL PAYROLL ($1,000)']


#### Create joins to establish consolidated dataframe

In [102]:

business_df1 = pd.merge(business_df,naics_df,on='NAICS CODE',how='left')
business_df1= pd.merge(business_df1,states_df,on='STATE DESCRIPTION',how='left')
business_df1.head(1)


,STATE DESCRIPTION,NAICS CODE,ENTERPRISE EMPLOYMENT SIZE,NUMBER OF FIRMS,NUMBER OF ESTABLISHMENTS,EMPLOYMENT,YEAR,ANNUAL PAYROLL,NAME,STATE CODE
0,Alabama,--,Total,79812,103875,1714692,2008,59827325000,Total,AL


#### Reorder Dataframe

In [103]:

business_df1 = business_df1[['YEAR','STATE DESCRIPTION', 'STATE CODE', 'NAICS CODE', 'NAME', 'ENTERPRISE EMPLOYMENT SIZE', 'NUMBER OF FIRMS', 'NUMBER OF ESTABLISHMENTS', 'EMPLOYMENT', 'ANNUAL PAYROLL']].copy()
business_df1.head(0)


,YEAR,STATE DESCRIPTION,STATE CODE,NAICS CODE,NAME,ENTERPRISE EMPLOYMENT SIZE,NUMBER OF FIRMS,NUMBER OF ESTABLISHMENTS,EMPLOYMENT,ANNUAL PAYROLL


#### Table with all relavent data to be jsonified

In [104]:

table_df = business_df1.rename(columns = {'STATE DESCRIPTION': 'STATE_DESCRIPTION', 'STATE CODE' : 'STATE_CODE', 'NAICS CODE' : 'NAICS_CODE' , 'ENTERPRISE EMPLOYMENT SIZE' : 'ENTERPRISE_EMPLOYMENT_SIZE', 'NUMBER OF FIRMS': 'NUMBER_OF_FIRMS', 'NUMBER OF ESTABLISHMENTS' : 'NUMBER_OF_ESTABLISHMENTS', 'ANNUAL PAYROLL' : 'ANNUAL_PAYROLL' })
table_df.head(0)


,YEAR,STATE_DESCRIPTION,STATE_CODE,NAICS_CODE,NAME,ENTERPRISE_EMPLOYMENT_SIZE,NUMBER_OF_FIRMS,NUMBER_OF_ESTABLISHMENTS,EMPLOYMENT,ANNUAL_PAYROLL
